In [ ]:
# ==============================================================================
# Step 1: Import Libraries
# ==============================================================================
print("Importing necessary libraries...")
import pandas as pd
import numpy as np
import warnings
import re

# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore')

# Import models
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb

# Import metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
print("Libraries imported successfully.")


# ==============================================================================
# Step 2: Load Your CSV Data
# ==============================================================================
# This script assumes your file is named 'latest water future.csv'
file_name = 'latest water future.csv'
print(f"Loading the data file: {file_name}")

try:
    df = pd.read_csv(file_name)
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"---! ERROR !---")
    print(f"The file '{file_name}' was not found in your Colab environment.")
    print("Please make sure you have uploaded the file and the name is spelled correctly.")
    raise


# ==============================================================================
# Step 3: Data Cleaning and Preprocessing
# ==============================================================================
print("\nStarting data cleaning and preprocessing...")

# Standardize column names
rename_mapping = {
    'dissolvedo': 'dissolved_oxygen',
    'dissolvedoxygen': 'dissolved_oxygen',
    'FecalColifo': 'FecalColiform'
}
df.rename(columns=rename_mapping, inplace=True)

# Convert sample_date to datetime format
df['sample_date'] = pd.to_datetime(df['sample_date'])

# Clean Station Names
df['Station Name'] = df['Station Name'].str.strip()
df['Station Name'] = df['Station Name'].str.replace(r'\s+', ' ', regex=True)

# Clean Season column
if 'Season' in df.columns:
    df['Season'] = df['Season'].str.strip()

# Convert all relevant columns to numeric
cols_to_numeric = ['pH', 'dissolved_oxygen', 'bod', 'cod', 'nitrate', 'FecalColiform', 'wqi']
for col in cols_to_numeric:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

df.dropna(subset=['wqi'], inplace=True)
df = df[df['wqi'] > 0]
df = df.sort_values(by=['Station Name', 'sample_date']).reset_index(drop=True)

print("Data cleaning finished. Shape of cleaned data:", df.shape)


# ==============================================================================
# Step 4: Feature Engineering
# ==============================================================================
print("\nStarting feature engineering...")

# Create time-based features
df['year'] = df['sample_date'].dt.year
df['month'] = df['sample_date'].dt.month

# Create lag and rolling features
for lag in [1, 2, 3]:
    df[f'wqi_lag_{lag}'] = df.groupby('Station Name')['wqi'].shift(lag)

for col in ['dissolved_oxygen', 'bod', 'cod', 'nitrate']:
    df[col] = df.groupby('Station Name')[col].transform(lambda x: x.ffill().bfill())
    df[f'{col}_roll_avg_3'] = df.groupby('Station Name')[col].shift(1).rolling(window=3, min_periods=1).mean()

# One-hot encode categorical columns
categorical_cols = ['Station Name']
if 'Season' in df.columns:
    categorical_cols.append('Season')

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
df.dropna(inplace=True)

print("Feature engineering complete. Shape of final data:", df.shape)


# ==============================================================================
# Step 5: Model Training and Evaluation
# ==============================================================================
print("\nPreparing data for modeling...")

# Define features (X) and target (y)
cols_to_drop = ['sample_date', 'wqi', 'latitude', 'longitude', 'wqi_index', 'Year', 'Month']
features = [col for col in df.columns if col not in cols_to_drop]
X = df[features]
y = df['wqi']

# Chronological split
train_mask = (df['sample_date'] < '2024-01-01')
test_mask = (df['sample_date'] >= '2024-01-01')
X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

# --- **FIX APPLIED HERE: Clean column names for special characters** ---
# LightGBM and XGBoost cannot handle special characters in feature names.
# This removes any characters that are not letters, numbers, or underscores.
X_train.columns = ["".join (c if c.isalnum() else '_' for c in str(x)) for x in X_train.columns]
X_test.columns = ["".join (c if c.isalnum() else '_' for c in str(x)) for x in X_test.columns]

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

# Define the models to train
models = {
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "LightGBM": lgb.LGBMRegressor(random_state=42),
    "XGBoost": xgb.XGBRegressor(random_state=42, objective='reg:squarederror')
}

# Train and evaluate each model
results = []
print("\nTraining and evaluating models...")
for name, model in models.items():
    print(f"--- Training {name} ---")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    results.append({"Model": name, "R-squared": r2, "MAE": mae, "RMSE": rmse})
    print(f"{name} evaluation complete.")

# ==============================================================================
# Step 6: Compare Model Performance
# ==============================================================================
print("\n--- Model Comparison ---")
results_df = pd.DataFrame(results).sort_values(by='R-squared', ascending=False).set_index('Model')
print(results_df.round(3))

best_model_name = results_df.index[0]
print(f"\n✅ Based on the R-squared score, the best model is: **{best_model_name}**")

Importing necessary libraries...
Libraries imported successfully.
Loading the data file: latest water future.csv
Data loaded successfully.

Starting data cleaning and preprocessing...
Data cleaning finished. Shape of cleaned data: (882, 15)

Starting feature engineering...
Feature engineering complete. Shape of final data: (831, 40)

Preparing data for modeling...
Training set size: 542 samples
Test set size: 289 samples

Training and evaluating models...
--- Training Random Forest ---
Random Forest evaluation complete.
--- Training LightGBM ---
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000214 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1383
[LightGBM] [Info] Number of data points in the train set: 542, number of used features: 33
[LightGBM] [Info] Start training from score 64.297380
[LightGBM] [Warning] No further splits with

In [ ]:
# ==============================================================================
# Step 1: Import Libraries
# ==============================================================================
print("Importing necessary libraries...")
import pandas as pd
import numpy as np
import warnings
import re

# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore')

# Import models
import lightgbm as lgb
print("Libraries imported successfully.")


# ==============================================================================
# Step 2: Load and Prepare the Full Dataset
# ==============================================================================
# This script assumes your file is named 'latest water future.csv'
file_name = 'latest water future.csv'
print(f"Loading and preparing the data file: {file_name}")

try:
    df = pd.read_csv(file_name)
except FileNotFoundError:
    print(f"---! ERROR !--- \nThe file '{file_name}' was not found. Please upload it and try again.")
    raise

# --- Data Cleaning ---
rename_mapping = {
    'dissolvedo': 'dissolved_oxygen',
    'dissolvedoxygen': 'dissolved_oxygen',
    'FecalColifo': 'FecalColiform'
}
df.rename(columns=rename_mapping, inplace=True)
df['sample_date'] = pd.to_datetime(df['sample_date'])
df['Station Name'] = df['Station Name'].str.strip().str.replace(r'\s+', ' ', regex=True)
if 'Season' in df.columns:
    df['Season'] = df['Season'].str.strip()
cols_to_numeric = ['pH', 'dissolved_oxygen', 'bod', 'cod', 'nitrate', 'FecalColiform', 'wqi']
for col in cols_to_numeric:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
df.dropna(subset=['wqi'], inplace=True)
df = df[df['wqi'] > 0]
df = df.sort_values(by=['Station Name', 'sample_date']).reset_index(drop=True)
print("Data cleaning finished.")

# --- Feature Engineering ---
print("Engineering features for the model...")
df['year'] = df['sample_date'].dt.year
df['month'] = df['sample_date'].dt.month
for lag in [1, 2, 3]:
    df[f'wqi_lag_{lag}'] = df.groupby('Station Name')['wqi'].shift(lag)
for col in ['dissolved_oxygen', 'bod', 'cod', 'nitrate']:
    df[col] = df.groupby('Station Name')[col].transform(lambda x: x.ffill().bfill())
    df[f'{col}_roll_avg_3'] = df.groupby('Station Name')[col].shift(1).rolling(window=3, min_periods=1).mean()

# Keep a list of original station names before encoding
original_station_names = df['Station Name'].unique()

# One-hot encode categorical columns
categorical_cols = ['Station Name']
if 'Season' in df.columns:
    categorical_cols.append('Season')
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
df_encoded.dropna(inplace=True)
print("Feature engineering complete.")

# ==============================================================================
# Step 3: Train the Final Model on ALL Available Data
# ==============================================================================
print("\nTraining the final model on the entire dataset...")

# Define the final set of features and target
cols_to_drop = ['sample_date', 'wqi', 'latitude', 'longitude', 'wqi_index', 'Year', 'Month']
features = [col for col in df_encoded.columns if col not in cols_to_drop]
X_full = df_encoded[features]
y_full = df_encoded['wqi']

# Clean feature names for LightGBM
X_full.columns = ["".join (c if c.isalnum() else '_' for c in str(x)) for x in X_full.columns]

# Train the final model
final_model = lgb.LGBMRegressor(random_state=42)
final_model.fit(X_full, y_full)
print("Final model has been trained successfully.")

# ==============================================================================
# Step 4: Iteratively Predict the Future
# ==============================================================================
print("\nGenerating future predictions month-by-month...")

# Use the original (un-encoded) dataframe as the basis for our history
history_df = df.copy()

# Define the dates we want to forecast
last_known_date = history_df['sample_date'].max()
future_dates = pd.date_range(start=last_known_date + pd.DateOffset(months=1), end='2025-12-31', freq='MS')

for date in future_dates:
    for station in original_station_names:
        # Get the last 3 months of history for the current station
        station_history = history_df[history_df['Station Name'] == station].tail(3)

        # Create a new row for the prediction
        new_row = {'sample_date': date, 'Station Name': station}

        # --- Engineer Features for the new row ---
        new_row['year'] = date.year
        new_row['month'] = date.month

        # Lag features from history
        new_row['wqi_lag_1'] = station_history['wqi'].iloc[-1]
        new_row['wqi_lag_2'] = station_history['wqi'].iloc[-2]
        new_row['wqi_lag_3'] = station_history['wqi'].iloc[-3]

        # For other parameters, we assume they stay the same as the last known value
        for param in ['pH', 'dissolved_oxygen', 'bod', 'cod', 'nitrate', 'FecalColiform', 'Season']:
             if param in station_history.columns:
                new_row[param] = station_history[param].iloc[-1]

        # Rolling averages from history
        for col in ['dissolved_oxygen', 'bod', 'cod', 'nitrate']:
            new_row[f'{col}_roll_avg_3'] = station_history[col].mean()

        # --- Predict ---
        # Convert the new row into a DataFrame
        new_row_df = pd.DataFrame([new_row])
        # One-hot encode this new row
        new_row_encoded = pd.get_dummies(new_row_df, columns=categorical_cols, drop_first=True)
        # Align columns with the training data to ensure format is identical
        new_row_aligned = new_row_encoded.reindex(columns=df_encoded.columns, fill_value=0)

        # Select features and clean column names
        X_future = new_row_aligned[features]
        X_future.columns = ["".join (c if c.isalnum() else '_' for c in str(x)) for x in X_future.columns]

        # Make the prediction
        prediction = final_model.predict(X_future)[0]

        # --- Update History ---
        # Add the prediction to the new row
        new_row['wqi'] = prediction
        # Append the complete new row to our history for the next iteration
        history_df = pd.concat([history_df, pd.DataFrame([new_row])], ignore_index=True)

# ==============================================================================
# Step 5: Display the Final Forecast
# ==============================================================================
print("\n--- Future WQI Forecast for 2025 ---")
# Filter only for the predicted future dates
predictions_df = history_df[history_df['sample_date'] >= future_dates[0]]

# Pivot the table for easy reading
forecast_table = predictions_df.pivot_table(
    values='wqi',
    index='Station Name',
    columns=predictions_df['sample_date'].dt.strftime('%Y-%m'),
    aggfunc='mean'
)

print(forecast_table.round(1))

Importing necessary libraries...
Libraries imported successfully.
Loading and preparing the data file: latest water future.csv
Data cleaning finished.
Engineering features for the model...
Feature engineering complete.

Training the final model on the entire dataset...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000191 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2000
[LightGBM] [Info] Number of data points in the train set: 831, number of used features: 33
[LightGBM] [Info] Start training from score 63.693033
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Final model has been trained successfully.

Generating future predictions month-by-month.

**MODEL TRAINING**

In [ ]:
# ==============================================================================
# Step 1: Import Libraries and Prepare Data
# ==============================================================================
print("Importing libraries and preparing data...")
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import r2_score

warnings.filterwarnings('ignore')

# Load the data from the file you uploaded
file_name = 'latest water future.csv'
try:
    df = pd.read_csv(file_name)
    print(f"Successfully loaded '{file_name}'")
except FileNotFoundError:
    print(f"---! ERROR !---")
    print(f"The file '{file_name}' was not found in your Colab environment.")
    print("Please make sure you have uploaded the file and the name is spelled correctly.")
    raise

# --- Data Cleaning ---
# Standardize column names that might be inconsistent
rename_mapping = {
    'dissolvedoxygen': 'dissolved_oxygen',
    'dissolvedo': 'dissolved_oxygen',
    'FecalColifo': 'FecalColiform'
}
df.rename(columns=rename_mapping, inplace=True)

# Clean date, station, and season columns
df['sample_date'] = pd.to_datetime(df['sample_date'])
df['Station Name'] = df['Station Name'].str.strip().str.replace(r'\s+', ' ', regex=True)
if 'Season' in df.columns:
    df['Season'] = df['Season'].str.strip()

# Convert data columns to numbers, forcing any errors into 'Not a Number' (NaN)
cols_to_numeric = ['pH', 'dissolved_oxygen', 'bod', 'cod', 'nitrate', 'wqi']
for col in cols_to_numeric:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Remove rows with missing or invalid WQI values
df.dropna(subset=['wqi'], inplace=True)
df = df[df['wqi'] > 0]
df = df.sort_values(by=['Station Name', 'sample_date']).reset_index(drop=True)
print("Data preparation complete.")


# ==============================================================================
# Step 2: Feature Engineering and Future Target Creation
# ==============================================================================
print("\nCreating features and future targets...")
df_model = df.copy()

# Add time-based features
df_model['year'] = df_model['sample_date'].dt.year
df_model['month'] = df_model['sample_date'].dt.month

# Convert categorical text columns into a numerical format
categorical_cols = ['Station Name']
if 'Season' in df_model.columns:
    categorical_cols.append('Season')
df_model = pd.get_dummies(df_model, columns=categorical_cols, drop_first=True)

# Create the future target columns (e.g., WQI in 1 month, 2 months, etc.)
horizons = [1, 2, 3, 4, 5, 6]
for h in horizons:
    # Group by the encoded station name columns to handle each station separately
    group_keys = df_model.filter(like='Station Name_').columns.tolist()
    df_model[f'wqi_{h}_month'] = df_model.groupby(group_keys)['wqi'].shift(-h)

# Drop any rows that now have missing future values
df_model.dropna(inplace=True)
print("Features and targets created successfully.")


# ==============================================================================
# Step 3: Train and Evaluate Multiple Models
# ==============================================================================
print("\nPreparing to train and compare multiple models...")
# Define our features (X) and our multiple targets (y)
features_to_drop = ['sample_date', 'wqi'] + [f'wqi_{h}_month' for h in horizons]
X = df_model.drop(columns=features_to_drop)
y = df_model[[f'wqi_{h}_month' for h in horizons]]

# Clean feature names to remove special characters for LightGBM/XGBoost
X.columns = ["".join (c if c.isalnum() else '_' for c in str(x)) for x in X.columns]

# Use a chronological split for time series data (last 20% is for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
print(f"Training on {len(X_train)} samples, testing on {len(X_test)} samples.")

# --- Define the models we want to compare ---
base_models = {
    "LightGBM": lgb.LGBMRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "XGBoost": xgb.XGBRegressor(random_state=42, objective='reg:squarederror')
}

# This dictionary will store the performance of each model
model_performance = {}

# --- Loop through each model to train and evaluate ---
for name, model in base_models.items():
    print(f"\n--- Training {name} ---")

    # Wrap the base model in MultiOutputRegressor to predict all 6 months at once
    multi_model = MultiOutputRegressor(model)
    multi_model.fit(X_train, y_train)

    # Get predictions for all future months
    y_pred = multi_model.predict(X_test)

    # Calculate R-squared for each future month
    horizon_scores = []
    for i, h in enumerate(horizons):
        r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
        horizon_scores.append(r2)

    model_performance[name] = horizon_scores
    print(f"{name} evaluation complete.")

# ==============================================================================
# Step 4: Display the Final Comparison Table
# ==============================================================================
print("\n--- Final Model Comparison ---")
performance_df = pd.DataFrame.from_dict(model_performance, orient='index')
performance_df.columns = [f'+{h} Month R²' for h in horizons]
performance_df.index.name = "Model"

# Calculate the average R² across all 6 months for an overall score
performance_df['Average R²'] = performance_df.mean(axis=1)
performance_df = performance_df.sort_values(by='Average R²', ascending=False)


print(performance_df.round(3))
best_model_name = performance_df.index[0]
print(f"\n✅ Based on the average R-squared score, the best model is: **{best_model_name}**")

Importing libraries and preparing data...
Successfully loaded 'latest water future.csv'
Data preparation complete.

Creating features and future targets...
Features and targets created successfully.

Preparing to train and compare multiple models...
Training on 623 samples, testing on 156 samples.

--- Training LightGBM ---
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000051 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 431
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 28
[LightGBM] [Info] Start training from score 65.646260
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("Starting EDA plot generation...")

# =============================================================================
# Step 1: Load and Clean the Data
# =============================================================================
file_name = 'latest water future.csv'
try:
    df = pd.read_csv(file_name)
    print(f"Successfully loaded '{file_name}'")
except FileNotFoundError:
    print(f"---! ERROR !---")
    print(f"The file '{file_name}' was not found.")
    print("Please make sure you have uploaded the file and the name is spelled correctly.")
    # In a real scenario, you'd stop here, but for this tool, we'll let it error out if it continues.
    raise

# --- Basic Data Cleaning (from your notebook) ---
# Standardize column names
rename_mapping = {
    'dissolvedo': 'dissolved_oxygen',
    'dissolvedoxygen': 'dissolved_oxygen',
    'FecalColifo': 'FecalColiform'
}
df.rename(columns=rename_mapping, inplace=True)

# Convert sample_date to datetime
df['sample_date'] = pd.to_datetime(df['sample_date'])

# Convert all relevant columns to numeric, forcing errors to NaN
cols_to_numeric = ['pH', 'dissolved_oxygen', 'bod', 'cod', 'nitrate', 'FecalColiform', 'wqi']
for col in cols_to_numeric:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows where key values are missing, as we can't plot them
df.dropna(subset=cols_to_numeric, inplace=True)
print("Data cleaning complete.")

# =============================================================================
# Plot 1: Distribution of Water Quality Index (WQI)
# =============================================================================
print("Generating Plot 1: WQI Distribution...")
plt.figure(figsize=(10, 6))
sns.histplot(df['wqi'], kde=True, bins=30, color='blue')
plt.title('Distribution of Water Quality Index (WQI)', fontsize=16, weight='bold')
plt.xlabel('WQI', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('wqi_distribution.png')
plt.clf()  # Clear the figure

# =============================================================================
# Plot 2: Distribution of Key Water Quality Parameters
# =============================================================================
print("Generating Plot 2: Parameter Distributions...")
parameters = ['pH', 'dissolved_oxygen', 'bod', 'cod', 'nitrate', 'FecalColiform']
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Distribution of Key Water Quality Parameters', fontsize=20, weight='bold')

# Flatten the axes array for easy iteration
for ax, param in zip(axes.flatten(), parameters):
    sns.histplot(df[param].dropna(), ax=ax, kde=True, bins=30)
    ax.set_title(f'Distribution of {param}', fontsize=14)
    ax.set_xlabel('Value', fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to make room for suptitle
plt.savefig('water_parameters_distribution.png')
plt.clf() # Clear the figure

# =============================================================================
# Plot 3: Water Quality Parameters Over Time (Monthly Average)
# =============================================================================
print("Generating Plot 3: Parameters Over Time...")
# We'll plot parameters with similar scales on different axes for clarity
time_df = df.set_index('sample_date')

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True)
fig.suptitle('Water Quality Parameters Over Time (Monthly Average)', fontsize=20, weight='bold')

# Axis 1: Parameters with smaller values
params_ax1 = ['pH', 'dissolved_oxygen', 'bod', 'cod', 'nitrate']
time_df[params_ax1].resample('MS').mean().plot(ax=ax1)
ax1.set_title('pH, Dissolved Oxygen, BOD, COD, Nitrate', fontsize=14)
ax1.set_ylabel('Average Value', fontsize=12)
ax1.legend(loc='upper right')
ax1.grid(linestyle='--', alpha=0.7)

# Axis 2: Parameters with larger values (Fecal Coliform)
params_ax2 = ['FecalColiform']
time_df[params_ax2].resample('MS').mean().plot(ax=ax2, color='purple')
ax2.set_title('Fecal Coliform', fontsize=14)
ax2.set_ylabel('Average Value', fontsize=12)
ax2.legend(loc='upper right')
ax2.grid(linestyle='--', alpha=0.7)

plt.xlabel('Date', fontsize=12)
plt.tight_layout(rect=[0, 0.03, 1, 0.96]) # Adjust layout
plt.savefig('water_parameters_over_time.png')
plt.clf() # Clear the figure

print("\nEDA plot generation complete! The following files have been saved:")
print("- wqi_distribution.png")
print("- water_parameters_distribution.png")
print("- water_parameters_over_time.png")

Starting EDA plot generation...
Successfully loaded 'latest water future.csv'
Data cleaning complete.
Generating Plot 1: WQI Distribution...
Generating Plot 2: Parameter Distributions...
Generating Plot 3: Parameters Over Time...

EDA plot generation complete! The following files have been saved:
- wqi_distribution.png
- water_parameters_distribution.png
- water_parameters_over_time.png


<Figure size 1000x600 with 0 Axes>

<Figure size 1800x1000 with 0 Axes>

<Figure size 1500x1200 with 0 Axes>

In [ ]:
# ==============================================================================
# Step 1: Import Libraries and Prepare Data
# ==============================================================================
print("Importing libraries and preparing data...")
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import r2_score

warnings.filterwarnings('ignore')

# Load the data from the file you uploaded
file_name = 'latest water future.csv'
try:
    df = pd.read_csv(file_name)
    print(f"Successfully loaded '{file_name}'")
except FileNotFoundError:
    print(f"---! ERROR !---")
    print(f"The file '{file_name}' was not found in your Colab environment.")
    print("Please make sure you have uploaded the file and the name is spelled correctly.")
    raise

# --- Data Cleaning ---
# Standardize column names that might be inconsistent
rename_mapping = {
    'dissolvedoxygen': 'dissolved_oxygen',
    'dissolvedo': 'dissolved_oxygen',
    'FecalColifo': 'FecalColiform'
}
df.rename(columns=rename_mapping, inplace=True)

# Clean date, station, and season columns
df['sample_date'] = pd.to_datetime(df['sample_date'])
df['Station Name'] = df['Station Name'].str.strip().str.replace(r'\s+', ' ', regex=True)
if 'Season' in df.columns:
    df['Season'] = df['Season'].str.strip()

# Convert data columns to numbers, forcing any errors into 'Not a Number' (NaN)
cols_to_numeric = ['pH', 'dissolved_oxygen', 'bod', 'cod', 'nitrate', 'wqi']
for col in cols_to_numeric:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Remove rows with missing or invalid WQI values
df.dropna(subset=['wqi'], inplace=True)
df = df[df['wqi'] > 0]
df = df.sort_values(by=['Station Name', 'sample_date']).reset_index(drop=True)
print("Data preparation complete.")


# ==============================================================================
# Step 2: Feature Engineering and Future Target Creation
# ==============================================================================
print("\nCreating features and future targets...")
df_model = df.copy()

# Add time-based features
df_model['year'] = df_model['sample_date'].dt.year
df_model['month'] = df_model['sample_date'].dt.month

# Convert categorical text columns into a numerical format
categorical_cols = ['Station Name']
if 'Season' in df_model.columns:
    categorical_cols.append('Season')
df_model = pd.get_dummies(df_model, columns=categorical_cols, drop_first=True)

# Create the future target columns (e.g., WQI in 1 month, 2 months, etc.)
horizons = [1, 2, 3, 4, 5, 6]
for h in horizons:
    # Group by the encoded station name columns to handle each station separately
    group_keys = df_model.filter(like='Station Name_').columns.tolist()
    df_model[f'wqi_{h}_month'] = df_model.groupby(group_keys)['wqi'].shift(-h)

# Drop any rows that now have missing future values
df_model.dropna(inplace=True)
print("Features and targets created successfully.")


# ==============================================================================
# Step 3: Train and Evaluate Multiple Models
# ==============================================================================
print("\nPreparing to train and compare multiple models...")
# Define our features (X) and our multiple targets (y)
features_to_drop = ['sample_date', 'wqi'] + [f'wqi_{h}_month' for h in horizons]
X = df_model.drop(columns=features_to_drop)
y = df_model[[f'wqi_{h}_month' for h in horizons]]

# Clean feature names to remove special characters for LightGBM/XGBoost
X.columns = ["".join (c if c.isalnum() else '_' for c in str(x)) for x in X.columns]

# Use a chronological split for time series data (last 20% is for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
print(f"Training on {len(X_train)} samples, testing on {len(X_test)} samples.")

# --- Define the models we want to compare ---
base_models = {
    "LightGBM": lgb.LGBMRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "XGBoost": xgb.XGBRegressor(random_state=42, objective='reg:squarederror')
}

# This dictionary will store the performance of each model
model_performance = {}

# --- Loop through each model to train and evaluate ---
for name, model in base_models.items():
    print(f"\n--- Training {name} ---")

    # Wrap the base model in MultiOutputRegressor to predict all 6 months at once
    multi_model = MultiOutputRegressor(model)
    multi_model.fit(X_train, y_train)

    # Get predictions for all future months
    y_pred = multi_model.predict(X_test)

    # Calculate R-squared for each future month
    horizon_scores = []
    for i, h in enumerate(horizons):
        r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
        horizon_scores.append(r2)

    model_performance[name] = horizon_scores
    print(f"{name} evaluation complete.")

# ==============================================================================
# Step 4: Display the Final Comparison Table
# ==============================================================================
print("\n--- Final Model Comparison ---")
performance_df = pd.DataFrame.from_dict(model_performance, orient='index')
performance_df.columns = [f'+{h} Month R²' for h in horizons]
performance_df.index.name = "Model"

# Calculate the average R² across all 6 months for an overall score
performance_df['Average R²'] = performance_df.mean(axis=1)
performance_df = performance_df.sort_values(by='Average R²', ascending=False)


print(performance_df.round(3))
best_model_name = performance_df.index[0]
print(f"\n✅ Based on the average R-squared score, the best model is: **{best_model_name}**")

Importing libraries and preparing data...
Successfully loaded 'latest water future.csv'
Data preparation complete.

Creating features and future targets...
Features and targets created successfully.

Preparing to train and compare multiple models...
Training on 623 samples, testing on 156 samples.

--- Training LightGBM ---
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000051 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 431
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 28
[LightGBM] [Info] Start training from score 65.646260
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

**MODEL COMPARISON**

In [ ]:
# ==============================================================================
# Step 1: Import Libraries and Prepare Data
# ==============================================================================
print("Importing libraries and preparing data...")
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler

# Import all models we want to test
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import r2_score

warnings.filterwarnings('ignore')

# Load the data
file_name = 'latest water future.csv'
df = pd.read_csv(file_name)

# --- Data Cleaning ---
rename_mapping = {'dissolvedoxygen': 'dissolved_oxygen'}
df.rename(columns=rename_mapping, inplace=True)
df['sample_date'] = pd.to_datetime(df['sample_date'])
df['Station Name'] = df['Station Name'].str.strip().str.replace(r'\s+', ' ', regex=True)
if 'Season' in df.columns:
    df['Season'] = df['Season'].str.strip()
cols_to_numeric = ['pH', 'dissolved_oxygen', 'bod', 'cod', 'nitrate', 'wqi']
for col in cols_to_numeric:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
df.dropna(subset=['wqi'], inplace=True)
df = df[df['wqi'] > 0]
df = df.sort_values(by=['Station Name', 'sample_date']).reset_index(drop=True)
print("Data preparation complete.")

# ==============================================================================
# Step 2: Feature Engineering and Target Creation
# ==============================================================================
print("\nCreating features and future targets...")
df_model = df.copy()

# Add time-based features
df_model['year'] = df_model['sample_date'].dt.year
df_model['month'] = df_model['sample_date'].dt.month

# Convert categorical text columns into a numerical format
categorical_cols = ['Station Name']
if 'Season' in df_model.columns:
    categorical_cols.append('Season')
df_model = pd.get_dummies(df_model, columns=categorical_cols, drop_first=True)

# Create future targets
horizons = [1, 2, 3, 4, 5, 6]
for h in horizons:
    group_keys = df_model.filter(like='Station Name_').columns.tolist()
    df_model[f'wqi_{h}_month'] = df_model.groupby(group_keys)['wqi'].shift(-h)

df_model.dropna(inplace=True)
print("Features and targets created successfully.")

# ==============================================================================
# Step 3: Train and Evaluate Multiple Models
# ==============================================================================
print("\nPreparing to train and compare multiple models...")
features_to_drop = ['sample_date', 'wqi'] + [f'wqi_{h}_month' for h in horizons]
X = df_model.drop(columns=features_to_drop)
y = df_model[[f'wqi_{h}_month' for h in horizons]]

X.columns = ["".join (c if c.isalnum() else '_' for c in str(x)) for x in X.columns]

# Chronological split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# --- NEW: Scale the features for SVR and MLP Regressor ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


print(f"Training on {len(X_train)} samples, testing on {len(X_test)} samples.")

# --- Define the models we want to compare ---
base_models = {
    "LightGBM": lgb.LGBMRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "XGBoost": xgb.XGBRegressor(random_state=42, objective='reg:squarederror'),
    "SVR": SVR(),
    "MLP Regressor": MLPRegressor(random_state=42, max_iter=500)
}

model_performance = {}

# --- Loop through each model to train and evaluate ---
for name, model in base_models.items():
    print(f"\n--- Training {name} ---")

    # Wrap the base model in MultiOutputRegressor
    multi_model = MultiOutputRegressor(model)

    # Use scaled data for SVR and MLP, and original data for tree-based models
    if name in ["SVR", "MLP Regressor"]:
        multi_model.fit(X_train_scaled, y_train)
        y_pred = multi_model.predict(X_test_scaled)
    else:
        multi_model.fit(X_train, y_train)
        y_pred = multi_model.predict(X_test)

    # Calculate R-squared for each horizon
    horizon_scores = []
    for i, h in enumerate(horizons):
        r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
        horizon_scores.append(r2)

    model_performance[name] = horizon_scores
    print(f"{name} evaluation complete.")

# ==============================================================================
# Step 4: Display the Final Comparison Table
# ==============================================================================
print("\n--- Final Model Comparison ---")
performance_df = pd.DataFrame.from_dict(model_performance, orient='index')
performance_df.columns = [f'+{h} Month R²' for h in horizons]
performance_df.index.name = "Model"

performance_df['Average R²'] = performance_df.mean(axis=1)
performance_df = performance_df.sort_values(by='Average R²', ascending=False)

print(performance_df.round(3))
best_model_name = performance_df.index[0]
print(f"\n✅ Based on the average R-squared score, the best model is: **{best_model_name}**")

Importing libraries and preparing data...
Data preparation complete.

Creating features and future targets...
Features and targets created successfully.

Preparing to train and compare multiple models...
Training on 623 samples, testing on 156 samples.

--- Training LightGBM ---
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000051 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 431
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 28
[LightGBM] [Info] Start training from score 65.646260
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn